# Medical Image Classification

**Company Use Case:**

**Approach:**

The following Notebook will show 3 approaches to do this with ConcreteML.

**Dataset Source:**

The data used is provided by MedMNIST v2, a comprehensive collection of standardized biomedical images. It encompasses 12 datasets for 2D and 6 for 3D images, pre-processed into 28 x 28 (2D) or 28 x 28 x 28 (3D) with corresponding classification labels. With 708,069 2D images and 9,998 3D images, it supports various classification tasks, from binary/multi-class to ordinal regression and multi-label, catering to biomedical image analysis, computer vision, and machine learning research and education.

https://medmnist.com/

**Dataset  1:**

_PneumoniaMNIST_

MedMNIST Description:

The PneumoniaMNIST is based on a prior dataset of 5,856 pediatric chest X-Ray images. The task is binary-class classification of pneumonia against normal. We split the source training set with a ratio of 9:1 into training and validation set and use its source validation set as the test set. The source images are gray-scale, and their sizes are (384−2,916)×(127−2,713). We center-crop the images and resize them into 1×28×28.

https://zenodo.org/records/6496656/files/pneumoniamnist.npz?download=1

**Dataset 2:**

_BreastMNIST_

MedMNIST Description:

The BreastMNIST is based on a dataset of 780 breast ultrasound images. It is categorized into 3 classes: normal, benign, and malignant. As we use low-resolution images, we simplify the task into binary classification by combining normal and benign as positive and classifying them against malignant as negative. We split the source dataset with a ratio of 7:1:2 into training, validation and test set. The source images of 1×500×500 are resized into 1×28×28.

https://zenodo.org/records/6496656/files/breastmnist.npz?download=1

**Dataset 3:**

_OrganCMNIST_

MedMNIST Description:

The OrganCMNIST is based on 3D computed tomography (CT) images from Liver Tumor Segmentation Benchmark (LiTS). It is renamed from OrganMNIST_Coronal (in MedMNIST v1) for simplicity. We use bounding-box annotations of 11 body organs from another study to obtain the organ labels. Hounsfield-Unit (HU) of the 3D images are transformed into gray-scale with an abdominal window. We crop 2D images from the center slices of the 3D bounding boxes in coronal views (planes). The images are resized into 1×28×28 to perform multi-class classification of 11 body organs. 115 and 16 CT scans from the source training set are used as training and validation set, respectively. The 70 CT scans from the source test set are treated as the test set.

https://zenodo.org/records/6496656/files/organcmnist.npz?download=1

# FHE Mode

In [13]:
mode = 'simulate'

# mode = 'execute'

# Imports

In [14]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import clear_output

# MedMNIST
import medmnist

# SciKit-Learn
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import *

# Imblearn
from imblearn.over_sampling import SMOTE

# XGBoost
from xgboost.sklearn import XGBClassifier

# PyTorch
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet50
from torchsummary import summary

# ConcreteML
from concrete.ml.sklearn import XGBClassifier as ConcreteXGBClassifier
from concrete.ml.sklearn import NeuralNetClassifier
from concrete.ml.torch.compile import compile_brevitas_qat_model

# Skorch
from skorch.callbacks import EpochScoring

# Brevitas
import brevitas

In [ ]:
# pep8 linting
%load_ext pycodestyle_magic
%pycodestyle_on

In [15]:
# set random seed
np.random.seed(1)
torch.manual_seed(1)

# General Functions

In [ ]:
################################################################################
# Load Data
################################################################################

def load_data(dataset, smote=True):

    '''
    Load data from MedMNIST

    Input:
        dataset (str): name of dataset

    Output:
        X_grayscale (tuple): (xtrain, xval, xtest) grayscale images
        X_rgb (tuple): (xtrain, xval, xtest) rgb images
        y (tuple): (ytrain, yval, ytest) labels
    '''

    # initialize DataClass
    DataClass = getattr(medmnist, dataset)
    # download data
    train_dataset = DataClass(split='train',download=True)
    eval_dataset = DataClass(split='val', download=True)
    test_dataset = DataClass(split='test', download=True)
    # to numpy array
    x = np.concatenate((train_dataset.imgs, eval_dataset.imgs, test_dataset.imgs), axis=0)
    y = np.concatenate((train_dataset.labels, eval_dataset.labels, test_dataset.labels), axis=0).flatten()
    # split into train, val, test=100 images
    xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=100, stratify=y)
    xtrain, xval, ytrain, yval = train_test_split(xtrain, ytrain, test_size=0.2, stratify=ytrain)
    # Smote
    # if smote:
    #     smote = SMOTE()
    #     xtrain, ytrain = smote.fit_resample(xtrain.reshape(xtrain.shape[0], -1), ytrain)
    #     xtrain = xtrain.reshape(xtrain.shape[0], 28, 28)
    # expand dimension (grayscale)
    # (channels=1, height=28, width=28)
    xtrain = np.expand_dims(xtrain, axis=1)
    xval = np.expand_dims(xval, axis=1)
    xtest = np.expand_dims(xtest, axis=1)

    clear_output()

    return xtrain, xval, xtest, ytrain, yval, ytest

################################################################################
# Preprocess Data
################################################################################

def plot_images(x_train, y_train, dataset):

    # plot sample images of each class
    n_classes = len(np.unique(y_train))

    fig, ax = plt.subplots(nrows=1, ncols=n_classes, figsize=(10, 5))
    for i in range(n_classes):
        ax[i].imshow(x_train[y_train.flatten()==i][0][0], cmap='gray')
        ax[i].set_title(f'Class {i}')
        ax[i].axis('off')
    fig.suptitle(dataset)
    plt.show()

################################################################################
# Plot Classification Metrics
################################################################################

def plot_classification_metrics(
    y_true: np.array,
    y_pred: np.array,
    plot_title: str = None
    ):

    '''
    Plots Classification Metrics

    Input:
      y_true = ground truth labels
      y_pred = prediction labels
      plot_title = title for results plot (optional)
    '''

    fig, ax = plt.subplots(2, 1, figsize=(5, 5))
    ax = ax.flatten()

    if len(np.unique(y_true)) == 2:
        accuracy = round(accuracy_score(y_true, y_pred),2)
        precision = round(precision_score(y_true, y_pred),2)
        recall = round(recall_score(y_true, y_pred),2)
        f1 = round(f1_score(y_true, y_pred),2)
        roc_auc = round(roc_auc_score(y_true, y_pred), 2)
        ax[0].bar(['Accuracy', 'Precision', 'Recall', 'F1', 'RocAuc'], [accuracy, precision, recall, f1, roc_auc])
    else:
        accuracy = round(accuracy_score(y_true, y_pred),2)
        precision = round(precision_score(y_true, y_pred, average='macro'),2)
        recall = round(recall_score(y_true, y_pred, average='macro'),2)
        f1 = round(f1_score(y_true, y_pred, average='macro'),2)
        ax[0].bar(['Accuracy', 'Precision', 'Recall', 'F1'], [accuracy, precision, recall, f1])


    # barchart of metrics for each classifier
    ax[0].set_title('Classifier Metrics')
    ax[0].set_ylim(0,1)
    ax[0].bar_label(ax[0].containers[0], label_type='center')

    # confusion matrix for each classifier
    cm = confusion_matrix(y_true, y_pred)
    ConfusionMatrixDisplay(cm).plot(ax=ax[1], cmap='Blues', colorbar=False)
    ax[1].set_title('Classifier Confusion Matrix')

    plt.suptitle(plot_title)
    plt.tight_layout()

    plt.show()

# Load Data

In [ ]:
data = load_data(dataset='PneumoniaMNIST')

xtrain_pneumonia, xval_pneumonia, xtest_pneumonia, ytrain_pneumonia, yval_pneumonia, ytest_pneumonia = data

data = load_data(dataset='BreastMNIST')

xtrain_breast, xval_breast, xtest_breast, ytrain_breast, yval_breast, ytest_breast = data

data = load_data(dataset='OrganCMNIST')

xtrain_organ, xval_organ, xtest_organ, ytrain_organ, yval_organ, ytest_organ = data

In [ ]:
print(f'Grayscale Shape PneumoniaMNIST: {xtrain_pneumonia.shape}')
print('\n')
print(f'Grayscale Shape BreastMNIST: {xtrain_breast.shape}')
print('\n')
print(f'Grayscale Shape OrganCMNIST: {xtrain_organ.shape}')

In [ ]:
plot_images(xtrain_pneumonia, ytrain_pneumonia, dataset='PneumoniaMNIST')
plot_images(xtrain_breast, ytrain_breast, dataset='BreastMNIST')
plot_images(xtrain_organ, ytrain_organ, dataset='OrganCMNIST')

# Preprocessing

In [ ]:
# normalize grayscale images for generic CNN

# PneumoniaMNIST
xtrain_pneumonia, xval_pneumonia, xtest_pneumonia = xtrain_pneumonia/255, xval_pneumonia/255, xtest_pneumonia/255

# BreastMNIST
xtrain_breast, xval_breast, xtest_breast = xtrain_breast/255, xval_breast/255, xtest_breast/255

# OrganCMNIST
xtrain_organ, xval_organ, xtest_organ = xtrain_organ/255, xval_organ/255, xtest_organ/255

# Modelling

## Approach 1: Artificial Neural Network

### Scikit-Learn

In [ ]:
class SklearnANN():

    def __init__(self):
        self.model = None
        self.log = {
            'train': 0,
            'evaluate_total': None,
            'evaluate_sample': None
        }

    def train(self, x_train, y_train, x_val, y_val, epochs=100, learning_rate=0.0001, weight_decay=0.0001, n_layers=2, neuron_factor=0.5, plot=True):
        # reshape data
        x_train, x_val = x_train.reshape(x_train.shape[0], -1), x_val.reshape(x_val.shape[0], -1)
        # initialize model
        self.model = MLPClassifier(
            hidden_layer_sizes=tuple([int(x_train.shape[1] * neuron_factor) for i in range(n_layers)]),
            learning_rate='adaptive',
            learning_rate_init=learning_rate,
            alpha=weight_decay,
            batch_size=128
        )
        # train model
        train_l, val_l, train_score, val_score = [], [], [], []
        for epoch in tqdm(range(epochs)):
            start_time = time.time()
            self.model.partial_fit(x_train, y_train, classes=np.unique(y_train))
            self.log['train'] += time.time() - start_time
            # curve metrics
            pred_train, pred_val  = self.model.predict_proba(x_train), self.model.predict_proba(x_val)
            train_l.append(log_loss(y_train, pred_train, labels=np.unique(y_train)))
            val_l.append(log_loss(y_val, pred_val, labels=np.unique(y_val)))
            train_score.append(accuracy_score(y_train, np.argmax(pred_train, axis=1)))
            val_score.append(accuracy_score(y_val, np.argmax(pred_val, axis=1)))

        # plot results
        if plot:
            fig, ax = plt.subplots(1, 2, figsize=(10, 5))
            ax[0].plot(train_l, label='Training Loss')
            ax[0].plot(val_l, label='Validation Loss')
            ax[0].set_xlabel('Epochs')
            ax[0].set_title('Loss')
            ax[0].legend()
            ax[1].plot(train_score, label='Training Accuracy')
            ax[1].plot(val_score, label='Validation Accuracy')
            ax[1].set_xlabel('Epochs')
            ax[1].set_title('Accuracy')
            ax[1].legend()
            plt.show()

        return self

    def evaluate(self, X):
        # reshape data for sklearn
        X = X.reshape(X.shape[0], -1)
        # predict
        start_time = time.time()
        y_pred = self.model.predict(X)
        self.log['evaluate_total'] = time.time() - start_time
        self.log['evaluate_sample'] = self.log['evaluate_total']/X.shape[0]

        return y_pred

#### PneumoniaMNIST

In [ ]:
# initialize model
sklearn_ann = SklearnANN()

# train model
sklearn_ann.train(
    x_train=xtrain_pneumonia,
    y_train=ytrain_pneumonia,
    x_val=xval_pneumonia,
    y_val=yval_pneumonia,
    epochs=50,
    learning_rate=0.0001,
    weight_decay=0.0001,
    n_layers=2,
    neuron_factor=0.5,
    plot=True
    )

# evaluate model
ypred_ann_sklearn = sklearn_ann.evaluate(xtest_pneumonia)

In [ ]:
plot_classification_metrics(y_true=ytest_pneumonia, y_pred=ypred_ann_sklearn, plot_title='Sklearn ANN')

In [ ]:
sklearn_results = pd.DataFrame(sklearn_ann.log, index=[0])
sklearn_results

#### BreastMNIST

In [ ]:
28*28*0.025

In [ ]:
# initialize model
sklearn_ann = SklearnANN()

# train model
sklearn_ann.train(
    x_train=xtrain_breast,
    y_train=ytrain_breast,
    x_val=xval_breast,
    y_val=yval_breast,
    epochs=50,
    learning_rate=0.0001,
    weight_decay=0.0001,
    n_layers=2,
    neuron_factor=0.5,
    plot=True
    )

# evaluate model
ypred_ann_sklearn = sklearn_ann.evaluate(xtest_breast)

In [ ]:
plot_classification_metrics(y_true=ytest_breast, y_pred=ypred_ann_sklearn, plot_title='Sklearn ANN')

In [ ]:
sklearn_results = pd.DataFrame(sklearn_ann.log, index=[0])
sklearn_results

#### OrganCMNIST

In [ ]:
# initialize model
sklearn_ann = SklearnANN()

# train model
sklearn_ann.train(
    x_train=xtrain_organ,
    y_train=ytrain_organ,
    x_val=xval_organ,
    y_val=yval_organ,
    epochs=50,
    learning_rate=0.0001,
    weight_decay=0.0001,
    n_layers=2,
    neuron_factor=0.5,
    plot=True
    )

# evaluate model
ypred_ann_sklearn = sklearn_ann.evaluate(xtest_organ)

In [ ]:
plot_classification_metrics(y_true=ytest_organ, y_pred=ypred_ann_sklearn, plot_title='Sklearn ANN')

In [ ]:
sklearn_results = pd.DataFrame(sklearn_ann.log, index=[0])
sklearn_results

### Concrete

In [ ]:
class ConcreteANN():
    
    def __init__(self, n_bits=2):
        self.n_bits = n_bits
        self.model = None
        self.fhe_circuit = None
        self.log = {
            'train': 0,
            'compile': None,
            'keygen': None,
            'evaluate_total': None,
            'evaluate_sample': None
        }

    def train(self, x_train, y_train, x_val, y_val, epochs=100, learning_rate=0.0001, weight_decay=0.0001, n_layers=2, neuron_factor=0.5, plot=True):
        # combine and reshape data
        x_train = np.concatenate((x_train, x_val), axis=0)
        x_train = x_train.reshape(x_train.shape[0], -1)
        y_train = np.concatenate((y_train, y_val), axis=0).astype(np.int64)

        # initialize model
        self.model = NeuralNetClassifier(
            lr=learning_rate,
            max_epochs=epochs,
            batch_size=128,
            callbacks=[EpochScoring(scoring='accuracy', name='train_acc', on_train=True)],
            verbose=0,
            **{
                'module__n_layers': n_layers,
                'module__n_w_bits': self.n_bits,
                'module__n_a_bits': self.n_bits,
                'module__n_accum_bits': 16,
                'module__n_hidden_neurons_multiplier': neuron_factor,
                'optimizer__weight_decay': weight_decay,
            }
        )
        # train model
        start_time = time.time()
        self.model.fit(X=x_train, y=y_train)
        self.log['train'] = time.time() - start_time
        clear_output()
        # compile model
        start_time = time.time()
        self.fhe_circuit = self.model.compile(x_train[:100])
        self.log['compile'] = time.time() - start_time

        # curve metrics
        train_l = self.model.sklearn_model.history[:, 'train_loss']
        val_l = self.model.sklearn_model.history[:, 'valid_loss']
        train_score = self.model.sklearn_model.history[:, 'train_acc']
        val_score = self.model.sklearn_model.history[:, 'valid_acc']
        # plot results
        if plot:
            fig, ax = plt.subplots(1, 2, figsize=(10, 5))
            ax[0].plot(train_l, label='Training Loss')
            ax[0].plot(val_l, label='Validation Loss')
            ax[0].set_xlabel('Epochs')
            ax[0].set_title('Loss')
            ax[0].legend()
            ax[1].plot(train_score, label='Training Accuracy')
            ax[1].plot(val_score, label='Validation Accuracy')
            ax[1]. set_xlabel('Epochs')
            ax[1].set_title('Accuracy')
            ax[1].legend()
            plt.show()

        return self
    
    def evaluate(self, X, fhe='simulate'):
        # reshape data
        X = X.reshape(X.shape[0], -1)
        # key generation
        start_time = time.time()
        self.fhe_circuit.keygen(force=True)
        self.log['keygen'] = time.time() - start_time
        # predict
        start_time = time.time()
        y_pred = np.array([self.model.predict(X[[i]], fhe=fhe)[0] for i in tqdm(range(X.shape[0]))])
        self.log['evaluate_total'] = time.time() - start_time
        self.log['evaluate_sample'] = self.log['evaluate_total']/X.shape[0]

        return y_pred

#### PneumoniaMNIST

In [ ]:
# initialize model
concrete_ann = ConcreteANN(n_bits=2)

# train model
concrete_ann.train(
    x_train=xtrain_pneumonia,
    y_train=ytrain_pneumonia,
    x_val=xval_pneumonia,
    y_val=yval_pneumonia,
    epochs=50,
    learning_rate=0.0001,
    weight_decay=0.0001,
    n_layers=2,
    neuron_factor=0.5,
    plot=True
    )

# evaluate model
ypred_ann_concrete = concrete_ann.evaluate(xtest_pneumonia, fhe=mode)

In [ ]:
plot_classification_metrics(y_true=ytest_pneumonia, y_pred=ypred_ann_concrete, plot_title='Concrete ANN (2-bit)')

In [ ]:
results = {'n_bits': [], 'y_pred': [], 'times': [], 'metrics': []}

for n_bits in range(2,7):
    
    concrete_ann = ConcreteANN(n_bits=n_bits)
    concrete_ann.train(x_train=xtrain_pneumonia, y_train=ytrain_pneumonia, x_val=xval_pneumonia, y_val=yval_pneumonia, epochs=50, learning_rate=0.0001, weight_decay=0.0001, n_layers=2, neuron_factor=0.5, plot=False)
    ypred_concrete = concrete_ann.evaluate(xtest_pneumonia, fhe=mode)
    metrics = {
        'accuracy':   accuracy_score(ytest_pneumonia, ypred_concrete),
        'precision':  precision_score(ytest_pneumonia, ypred_concrete),
        'recall':     recall_score(ytest_pneumonia, ypred_concrete),
        'f1':         f1_score(ytest_pneumonia, ypred_concrete)
    }

    results['n_bits'].append(n_bits)
    results['y_pred'].append(ypred_concrete)
    results['times'].append(concrete_ann.log)
    results['metrics'].append(metrics)

In [ ]:
for preds, bits in zip(results['y_pred'], results['n_bits']):
    print('#################################################################')
    plot_classification_metrics(ytest_pneumonia, preds, plot_title=f'Quantization (n_bits={bits})')

In [ ]:
concrete_results = pd.concat([pd.DataFrame.from_records(results['times']), pd.DataFrame.from_records(results['metrics'])], axis=1)
concrete_results.insert(0, 'n_bits', results['n_bits'])
concrete_results

#### BreastMNIST

In [ ]:
# initialize model
concrete_ann = ConcreteANN(n_bits=2)

# train model
concrete_ann.train(
    x_train=xtrain_breast,
    y_train=ytrain_breast,
    x_val=xval_breast,
    y_val=yval_breast,
    epochs=50,
    learning_rate=0.0001,
    weight_decay=0.0001,
    n_layers=2,
    neuron_factor=0.5,
    plot=True
    )

# evaluate model
ypred_ann_concrete = concrete_ann.evaluate(xtest_breast, fhe=mode)

In [ ]:
plot_classification_metrics(y_true=ytest_breast, y_pred=ypred_ann_concrete, plot_title='Concrete ANN (2 n_bits)')

In [ ]:
results = {'n_bits': [], 'y_pred': [], 'times': [], 'metrics': []}

for n_bits in range(2,7):

    concrete_ann = ConcreteANN(n_bits=n_bits)
    concrete_ann.train(x_train=xtrain_breast, y_train=ytrain_breast, x_val=xval_breast, y_val=yval_breast, epochs=50, learning_rate=0.0001, weight_decay=0.0001, n_layers=2, neuron_factor=0.5, plot=False)
    ypred_concrete = concrete_ann.evaluate(xtest_breast, fhe=mode)
    metrics = {
        'accuracy':   accuracy_score(ytest_breast, ypred_concrete),
        'precision':  precision_score(ytest_breast, ypred_concrete),
        'recall':     recall_score(ytest_breast, ypred_concrete),
        'f1':         f1_score(ytest_breast, ypred_concrete)
    }

    results['n_bits'].append(n_bits)
    results['y_pred'].append(ypred_concrete)
    results['times'].append(concrete_ann.log)
    results['metrics'].append(metrics)

In [ ]:
for preds, bits in zip(results['y_pred'], results['n_bits']):
    print('#################################################################')
    plot_classification_metrics(ytest_breast, preds, plot_title=f'Quantization (n_bits={bits})')

In [ ]:
concrete_results = pd.concat([pd.DataFrame.from_records(results['times']), pd.DataFrame.from_records(results['metrics'])], axis=1)
concrete_results.insert(0, 'n_bits', results['n_bits'])
concrete_results

#### OrganCMNIST

In [ ]:
# initialize model
concrete_ann = ConcreteANN(n_bits=2)

# train model
concrete_ann.train(
    x_train=xtrain_organ,
    y_train=ytrain_organ,
    x_val=xval_organ,
    y_val=yval_organ,
    epochs=50,
    learning_rate=0.0001,
    weight_decay=0.0001,
    n_layers=2,
    neuron_factor=0.5,
    plot=True
    )

# evaluate model
ypred_ann_concrete = concrete_ann.evaluate(xtest_organ, fhe=mode)

In [ ]:
plot_classification_metrics(y_true=ytest_organ, y_pred=ypred_ann_concrete, plot_title='Concrete ANN (2 n_bits)')

In [ ]:
results = {'n_bits': [], 'y_pred': [], 'times': [], 'metrics': []}

for n_bits in range(2,7):

    concrete_ann = ConcreteANN(n_bits=n_bits)
    concrete_ann.train(x_train=xtrain_organ, y_train=ytrain_organ, x_val=xval_organ, y_val=yval_organ, epochs=50, learning_rate=0.0001, weight_decay=0.0001, n_layers=2, neuron_factor=0.5, plot=False)
    ypred_concrete = concrete_ann.evaluate(xtest_organ, fhe=mode)
    metrics = {
        'accuracy':   accuracy_score(ytest_organ, ypred_concrete),
        'precision':  precision_score(ytest_organ, ypred_concrete, average='macro'),
        'recall':     recall_score(ytest_organ, ypred_concrete, average='macro'),
        'f1':         f1_score(ytest_organ, ypred_concrete, average='macro')
    }

    results['n_bits'].append(n_bits)
    results['y_pred'].append(ypred_concrete)
    results['times'].append(concrete_ann.log)
    results['metrics'].append(metrics)

In [ ]:
for preds, bits in zip(results['y_pred'], results['n_bits']):
    print('#################################################################')
    plot_classification_metrics(ytest_organ, preds, plot_title=f'Quantization (n_bits={bits})')

In [ ]:
concrete_results = pd.concat([pd.DataFrame.from_records(results['times']), pd.DataFrame.from_records(results['metrics'])], axis=1)
concrete_results.insert(0, 'n_bits', results['n_bits'])
concrete_results

## Approach 2: Convolutional Neural Network

### PyTorch

In [ ]:
class TorchCNN():
    def __init__(self, model):
        self.model = model
        self.log = {
            'train': 0,
            'evaluate_total': None,
            'evaluate_sample': None
        }

    def train(self, x_train, y_train, x_val, y_val, epochs=30, learning_rate=0.0001, weight_decay=0.0001, plot=True):
        # data to tensor
        x_train, y_train = torch.tensor(x_train, dtype=torch.float), torch.tensor(y_train, dtype=torch.long).flatten()
        x_val, y_val = torch.tensor(x_val, dtype=torch.float), torch.tensor(y_val, dtype=torch.long).flatten()
        # criterion
        criterion = torch.nn.CrossEntropyLoss()
        # optimizer
        optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate, weight_decay=weight_decay)
        # train loop
        train_l, val_l, train_score, val_score= [], [], [], []
        for epoch in tqdm(range(epochs)):

            # set model to train mode
            self.model.train()
            start_time = time.time()
            for i in range(0, len(x_train), 1):
                inputs, labels = x_train[i:i+1], y_train[i:i+1]
                # forward pass
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)
                # backward pass
                optimizer.zero_grad()
                loss.backward()
                # update weights
                optimizer.step()
            epoch_time = time.time() - start_time
            self.log['train'] += epoch_time

            # set model to eval mode
            self.model.eval()
            # calculate curve metrics
            with torch.no_grad():
                train_outputs = self.model(x_train)
                train_loss = criterion(train_outputs, y_train)
                predicted = torch.argmax(train_outputs, dim=1)
                train_accuracy = accuracy_score(predicted.detach().numpy(), y_train.detach().numpy())*100

                val_outputs = self.model(x_val)
                val_loss = criterion(val_outputs, y_val)
                predicted = torch.argmax(val_outputs, dim=1)
                val_accuracy = accuracy_score(predicted.detach().numpy(), y_val.detach().numpy())*100

            train_l.append(train_loss.item())
            val_l.append(val_loss.item())
            train_score.append(train_accuracy)
            val_score.append(val_accuracy)

        if plot:
            fig, ax = plt.subplots(1, 2, figsize=(10, 5))
            ax[0].plot(train_l, label='Training Loss')
            ax[0].plot(val_l, label='Validation Loss')
            ax[0].set_xlabel('Epochs')
            ax[0].set_title('Loss')
            ax[0].legend()
            ax[1].plot(train_score, label='Training Accuracy')
            ax[1].plot(val_score, label='Validation Accuracy')
            ax[1]. set_xlabel('Epochs')
            ax[1].set_title('Accuracy')
            ax[1].legend()
            plt.show()

        return self

    def evaluate(self, X):
        # data to tensor
        X = torch.tensor(X, dtype=torch.float)
        # set model to eval mode
        self.model.eval()
        # predict
        start_time = time.time()
        with torch.no_grad():
            # predict each sample individually
            outputs = [self.model(X[[i]]) for i in tqdm(range(X.shape[0]))]
        self.log['evaluate_total'] = time.time() - start_time
        self.log['evaluate_sample'] = self.log['evaluate_total']/X.shape[0]
        # get binary predictions
        y_pred = torch.argmax(torch.cat(outputs), dim=1).detach().numpy().flatten()

        return y_pred

#### PneumoniaMNIST

In [ ]:
n_classes = len(np.unique(ytrain_pneumonia))

torch_model = torch.nn.Sequential(
        torch.nn.Conv2d(in_channels=1, out_channels=2, kernel_size=5),
        torch.nn.ReLU(),
        torch.nn.AvgPool2d(kernel_size=2, stride=2),
        torch.nn.Conv2d(in_channels=2, out_channels=4, kernel_size=5),
        torch.nn.ReLU(),
        torch.nn.AvgPool2d(kernel_size=2, stride=2),
        torch.nn.Flatten(),
        torch.nn.Linear(in_features=64, out_features=n_classes, bias=True)
    )

summary(torch_model, input_size=(1, 28, 28))

In [ ]:
# initialize model
torch_cnn = TorchCNN(model=torch_model)

# train model
torch_cnn = torch_cnn.train(
    x_train=xtrain_pneumonia,
    y_train=ytrain_pneumonia,
    x_val=xval_pneumonia,
    y_val=yval_pneumonia,
    epochs=30,
    learning_rate=0.0001,
    weight_decay=0.0001
    )

# evaluate model
ypred_torch = torch_cnn.evaluate(xtest_pneumonia)

In [ ]:
plot_classification_metrics(ytest_pneumonia, ypred_torch, plot_title='Torch CNN')

In [ ]:
torch_results = pd.DataFrame(torch_cnn.log, index=[0])
torch_results

#### BreastMNIST

In [ ]:
n_classes = len(np.unique(ytrain_breast))

torch_model = torch.nn.Sequential(
        torch.nn.Conv2d(in_channels=1, out_channels=2, kernel_size=5),
        torch.nn.ReLU(),
        torch.nn.AvgPool2d(kernel_size=2, stride=2),
        torch.nn.Conv2d(in_channels=2, out_channels=4, kernel_size=5),
        torch.nn.ReLU(),
        torch.nn.AvgPool2d(kernel_size=2, stride=2),
        torch.nn.Flatten(),
        torch.nn.Linear(in_features=64, out_features=n_classes, bias=True)
    )

summary(torch_model, input_size=(1, 28, 28))

In [ ]:
# initialize model
torch_cnn = TorchCNN(model=torch_model)

# train model
torch_cnn = torch_cnn.train(
    x_train=xtrain_breast,
    y_train=ytrain_breast,
    x_val=xval_breast,
    y_val=yval_breast,
    epochs=30,
    learning_rate=0.0001,
    weight_decay=0.0001
    )

# evaluate model
ypred_torch = torch_cnn.evaluate(xtest_breast)

In [ ]:
plot_classification_metrics(ytest_breast, ypred_torch, plot_title='Torch CNN')

In [ ]:
torch_results = pd.DataFrame(torch_cnn.log, index=[0])
torch_results

#### OrganCMNIST

In [ ]:
n_classes = len(np.unique(ytrain_organ))

torch_model = torch.nn.Sequential(
        torch.nn.Conv2d(in_channels=1, out_channels=2, kernel_size=5),
        torch.nn.ReLU(),
        torch.nn.AvgPool2d(kernel_size=2, stride=2),
        torch.nn.Conv2d(in_channels=2, out_channels=4, kernel_size=5),
        torch.nn.ReLU(),
        torch.nn.AvgPool2d(kernel_size=2, stride=2),
        torch.nn.Flatten(),
        torch.nn.Linear(in_features=64, out_features=n_classes, bias=True)
    )

summary(torch_model, input_size=(1, 28, 28))

In [ ]:
# initialize model
torch_cnn = TorchCNN(model=torch_model)

# train model
torch_cnn = torch_cnn.train(
    x_train=xtrain_organ,
    y_train=ytrain_organ,
    x_val=xval_organ,
    y_val=yval_organ,
    epochs=30,
    learning_rate=0.0001,
    weight_decay=0.0001
    )

# evaluate model
ypred_torch = torch_cnn.evaluate(xtest_organ)

In [ ]:
plot_classification_metrics(ytest_organ, ypred_torch, plot_title='Torch CNN')

In [ ]:
torch_results = pd.DataFrame(torch_cnn.log, index=[0])
torch_results

### Concrete

In [ ]:
class ConcreteCNN():
    def __init__(self, model):
        self.model = model
        self.log = {
            'train': 0,
            'compile': None,
            'keygen': None,
            'evaluate_total': None,
            'evaluate_sample': None
        }

    def train(self, x_train, y_train, x_val, y_val, epochs=30, learning_rate=0.0001, weight_decay=0.0001, plot=True):
        # get number of classes
        n_classes = len(np.unique(y_train))
        # data to tensor
        x_train, y_train = torch.tensor(x_train, dtype=torch.float), torch.tensor(y_train, dtype=torch.long).flatten()
        x_val, y_val = torch.tensor(x_val, dtype=torch.float), torch.tensor(y_val, dtype=torch.long).flatten()
        # criterion
        criterion = torch.nn.CrossEntropyLoss()
        # optimizer
        optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate, weight_decay=weight_decay)
        # train loop
        train_l, val_l, train_score, val_score= [], [], [], []
        for epoch in tqdm(range(epochs)):

            # set model to train mode
            self.model.train()
            start_time = time.time()
            for i in range(0, len(x_train), 1):
                inputs, labels = x_train[i:i+1], y_train[i:i+1]
                # forward pass
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)
                # backward pass
                optimizer.zero_grad()
                loss.backward()
                # update weights
                optimizer.step()
            epoch_time = time.time() - start_time
            self.log['train'] += epoch_time

            # set model to eval mode
            self.model.eval()
            # calculate curve metrics
            with torch.no_grad():
                train_outputs = self.model(x_train)
                train_loss = criterion(train_outputs, y_train)
                predicted = torch.argmax(train_outputs, dim=1)
                train_accuracy = accuracy_score(predicted.detach().numpy(), y_train.detach().numpy())*100

                val_outputs = self.model(x_val)
                val_loss = criterion(val_outputs, y_val)
                predicted = torch.argmax(val_outputs, dim=1)
                val_accuracy = accuracy_score(predicted.detach().numpy(), y_val.detach().numpy())*100

            train_l.append(train_loss.item())
            val_l.append(val_loss.item())
            train_score.append(train_accuracy)
            val_score.append(val_accuracy)

        if plot:
            fig, ax = plt.subplots(1, 2, figsize=(10, 5))
            ax[0].plot(train_l, label='Training Loss')
            ax[0].plot(val_l, label='Validation Loss')
            ax[0].set_xlabel('Epochs')
            ax[0].set_title('Loss')
            ax[0].legend()
            ax[1].plot(train_score, label='Training Accuracy')
            ax[1].plot(val_score, label='Validation Accuracy')
            ax[1]. set_xlabel('Epochs')
            ax[1].set_title('Accuracy')
            ax[1].legend()
            plt.show()

        # compile model
        start_time = time.time()
        self.model = compile_brevitas_qat_model(self.model, x_train)
        self.log['compile'] = time.time() - start_time

        return self
    
    def evaluate(self, X, fhe='simulate'):
        # keygen
        start_time = time.time()
        self.model.fhe_circuit.keygen(force=True)
        self.log['keygen'] = time.time() - start_time
        # predict
        start_time = time.time()
        with torch.no_grad():
            # predict each sample individually
            outputs = [self.model.forward(X[[i]], fhe=fhe) for i in tqdm(range(X.shape[0]))]
        self.log['evaluate_total'] = time.time() - start_time
        self.log['evaluate_sample'] = self.log['evaluate_total']/X.shape[0]
        # get binary predictions
        y_pred = torch.argmax(torch.cat(outputs), dim=1).detach().numpy().flatten()

        return y_pred

#### PneumoniaMNIST

In [ ]:
results = {'n_bits': [], 'y_pred': [], 'times': [], 'metrics': []}

for n_bits in range(2,7):

    brevitas_model = torch.nn.Sequential(
    # entry point of a network has to be a QuantIdentity layer
    # to quantize the inputs
    brevitas.nn.QuantIdentity(bit_width=n_bits, return_quant_tensor=True),
    # brevitas QuantConv2d layer
    brevitas.nn.QuantConv2d(in_channels=1, out_channels=1, kernel_size=3, weight_bit_width=n_bits),
    # brevitas QuantReLU layer
    brevitas.nn.QuantReLU(bit_width=n_bits),
    # torch AvgPool2d layer
    torch.nn.AvgPool2d(kernel_size=2, stride=2),
    # after a PyTorch layer there needs to follow a QuantIdentity layer
    # to make sure the inputs are quantized correctly
    brevitas.nn.QuantIdentity(bit_width=n_bits, return_quant_tensor=True),
    # PyTorch flatten layer
    torch.nn.Flatten(),
    # again a QuantIdentity layer after the PyTorch layer
    brevitas.nn.QuantIdentity(bit_width=n_bits, return_quant_tensor=True),
    # brevitas QuantLinear (output) layer
    brevitas.nn.QuantLinear(in_features=169, out_features=2, bias=True, weight_bit_width=n_bits)
    )

    concrete_cnn = ConcreteCNN(model=brevitas_model)
    concrete_cnn = concrete_cnn.train(x_train=xtrain_pneumonia, y_train=ytrain_pneumonia, x_val=xval_pneumonia, y_val=yval_pneumonia, epochs=30, learning_rate=0.0001, weight_decay=0.0001, plot=False)
    ypred_concrete = concrete_cnn.evaluate(xtest_pneumonia, fhe=mode)
    metrics = {
        'accuracy':   accuracy_score(ytest_pneumonia, ypred_concrete),
        'precision':  precision_score(ytest_pneumonia, ypred_concrete),
        'recall':     recall_score(ytest_pneumonia, ypred_concrete),
        'f1':         f1_score(ytest_pneumonia, ypred_concrete)
    }

    results['n_bits'].append(n_bits)
    results['y_pred'].append(ypred_concrete)
    results['times'].append(concrete_cnn.log)
    results['metrics'].append(metrics)

In [ ]:
for preds, bits in zip(results['y_pred'], results['n_bits']):
    print('#################################################################')
    plot_classification_metrics(ytest_pneumonia, preds, plot_title=f'Quantization (n_bits={bits})')

In [ ]:
concrete_results = pd.concat([pd.DataFrame.from_records(results['times']), pd.DataFrame.from_records(results['metrics'])], axis=1)
concrete_results.insert(0, 'n_bits', results['n_bits'])
concrete_results

#### BreastMNIST

In [ ]:
results = {'n_bits': [], 'y_pred': [], 'times': [], 'metrics': []}

for n_bits in range(2,7):

    brevitas_model = torch.nn.Sequential(
    # entry point of a network has to be a QuantIdentity layer
    # to quantize the inputs
    brevitas.nn.QuantIdentity(bit_width=n_bits, return_quant_tensor=True),
    # brevitas QuantConv2d layer
    brevitas.nn.QuantConv2d(in_channels=1, out_channels=1, kernel_size=3, weight_bit_width=n_bits),
    # brevitas QuantReLU layer
    brevitas.nn.QuantReLU(bit_width=n_bits),
    # torch AvgPool2d layer
    torch.nn.AvgPool2d(kernel_size=2, stride=2),
    # after a PyTorch layer there needs to follow a QuantIdentity layer
    # to make sure the inputs are quantized correctly
    brevitas.nn.QuantIdentity(bit_width=n_bits, return_quant_tensor=True),
    # PyTorch flatten layer
    torch.nn.Flatten(),
    # again a QuantIdentity layer after the PyTorch layer
    brevitas.nn.QuantIdentity(bit_width=n_bits, return_quant_tensor=True),
    # brevitas QuantLinear (output) layer
    brevitas.nn.QuantLinear(in_features=169, out_features=2, bias=True, weight_bit_width=n_bits)
    )

    concrete_cnn = ConcreteCNN(model=brevitas_model)
    concrete_cnn = concrete_cnn.train(x_train=xtrain_breast, y_train=ytrain_breast, x_val=xval_breast, y_val=yval_breast, epochs=30, learning_rate=0.0001, weight_decay=0.0001, plot=False)
    ypred_concrete = concrete_cnn.evaluate(xtest_breast, fhe=mode)
    metrics = {
        'accuracy':   accuracy_score(ytest_breast, ypred_concrete),
        'precision':  precision_score(ytest_breast, ypred_concrete),
        'recall':     recall_score(ytest_breast, ypred_concrete),
        'f1':         f1_score(ytest_breast, ypred_concrete)
    }

    results['n_bits'].append(n_bits)
    results['y_pred'].append(ypred_concrete)
    results['times'].append(concrete_cnn.log)
    results['metrics'].append(metrics)

In [ ]:
for preds, bits in zip(results['y_pred'], results['n_bits']):
    print('#################################################################')
    plot_classification_metrics(ytest_breast, preds, plot_title=f'Quantization (n_bits={bits})')

In [ ]:
concrete_results = pd.concat([pd.DataFrame.from_records(results['times']), pd.DataFrame.from_records(results['metrics'])], axis=1)
concrete_results.insert(0, 'n_bits', results['n_bits'])
concrete_results

#### OrganCMNIST

In [ ]:
results = {'n_bits': [], 'y_pred': [], 'times': [], 'metrics': []}

for n_bits in range(2,7):
    
    brevitas_model = torch.nn.Sequential(
    # entry point of a network has to be a QuantIdentity layer
    # to quantize the inputs
    brevitas.nn.QuantIdentity(bit_width=n_bits, return_quant_tensor=True),
    # brevitas QuantConv2d layer
    brevitas.nn.QuantConv2d(in_channels=1, out_channels=1, kernel_size=3, weight_bit_width=n_bits),
    # brevitas QuantReLU layer
    brevitas.nn.QuantReLU(bit_width=n_bits),
    # torch AvgPool2d layer
    torch.nn.AvgPool2d(kernel_size=2, stride=2),
    # after a PyTorch layer there needs to follow a QuantIdentity layer
    # to make sure the inputs are quantized correctly
    brevitas.nn.QuantIdentity(bit_width=n_bits, return_quant_tensor=True),
    # PyTorch flatten layer
    torch.nn.Flatten(),
    # again a QuantIdentity layer after the PyTorch layer
    brevitas.nn.QuantIdentity(bit_width=n_bits, return_quant_tensor=True),
    # brevitas QuantLinear (output) layer
    brevitas.nn.QuantLinear(in_features=169, out_features=2, bias=True, weight_bit_width=n_bits)
    )

    concrete_cnn = ConcreteCNN(model=brevitas_model)
    concrete_cnn = concrete_cnn.train(x_train=xtrain_organ, y_train=ytrain_organ, x_val=xval_organ, y_val=yval_organ, epochs=30, learning_rate=0.0001, weight_decay=0.0001, plot=False)
    ypred_concrete = concrete_cnn.evaluate(xtest_organ, fhe=mode)
    metrics = {
        'accuracy':   accuracy_score(ytest_organ, ypred_concrete),
        'precision':  precision_score(ytest_organ, ypred_concrete, average='macro'),
        'recall':     recall_score(ytest_organ, ypred_concrete, average='macro'),
        'f1':         f1_score(ytest_organ, ypred_concrete, average='macro')
    }

    results['n_bits'].append(n_bits)
    results['y_pred'].append(ypred_concrete)
    results['times'].append(concrete_cnn.log)
    results['metrics'].append(metrics)

In [ ]:
for preds, bits in zip(results['y_pred'], results['n_bits']):
    print('#################################################################')
    plot_classification_metrics(ytest_organ, preds, plot_title=f'Quantization (n_bits={bits})')

In [ ]:
concrete_results = pd.concat([pd.DataFrame.from_records(results['times']), pd.DataFrame.from_records(results['metrics'])], axis=1)
concrete_results.insert(0, 'n_bits', results['n_bits'])
concrete_results

## Approach 3: Pre-Trained Model & ANN

### XGBoost

In [ ]:
class XGBPretrained():
    
    def __init__(self):
        self.model = None
        self.pretrained_model = None
        self.log = {
            'train': 0,
            'evaluate_total': None,
            'evaluate_sample': None
        }

    def train(self, x_train, y_train, x_val, y_val, n_estimators=100, max_depth=6, learning_rate=0.3):
        # transform data to tensor
        x_train, x_val = torch.tensor(x_train, dtype=torch.float), torch.tensor(x_val, dtype=torch.float)
        # combine data
        X, y = torch.cat((x_train, x_val), dim=0), np.concatenate((y_train, y_val), axis=0)
        # get resnet50 model
        self.pretrained_model = resnet50(pretrained=True)
        # modify first layer to accept grayscale images with one channel
        self.pretrained_model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        # remove last layer and replace with flattening layer
        self.pretrained_model.fc = torch.nn.Flatten()
        # feature extraction
        with torch.no_grad():
            X = self.pretrained_model(X).detach().numpy()
        # initialize XGBoost model
        self.model = XGBClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, n_jobs=-1)
        # train model
        start_time = time.time()
        self.model.fit(X, y)
        self.log['train'] = time.time() - start_time

        return self
    
    def evaluate(self, X):
        # transform data to tensor
        X = torch.tensor(X, dtype=torch.float)
        # feature extraction
        with torch.no_grad():
            X = self.pretrained_model(X).detach().numpy()
        # predict
        start_time = time.time()
        y_pred = self.model.predict(X)
        self.log['evaluate_total'] = time.time() - start_time
        self.log['evaluate_sample'] = self.log['evaluate_total']/X.shape[0]

        return y_pred

#### PneumoniaMNIST

In [ ]:
# initialize model
xgb_pretrained = XGBPretrained()

# train model
xgb_pretrained = xgb_pretrained.train(
    x_train=xtrain_pneumonia,
    y_train=ytrain_pneumonia,
    x_val=xval_pneumonia,
    y_val=yval_pneumonia,
    n_estimators=100,
    max_depth=6,
    learning_rate=0.3
    )

# evaluate model
ypred_xgb_pretrained = xgb_pretrained.evaluate(xtest_pneumonia)

In [ ]:
plot_classification_metrics(ytest_pneumonia, ypred_xgb_pretrained, plot_title='XGB Pretrained')

In [ ]:
xgb_results = pd.DataFrame(xgb_pretrained.log, index=[0])
xgb_results

#### BreastMNIST

In [ ]:
# initialize model
xgb_pretrained = XGBPretrained()

# train model
xgb_pretrained = xgb_pretrained.train(
    x_train=xtrain_breast,
    y_train=ytrain_breast,
    x_val=xval_breast,
    y_val=yval_breast,
    n_estimators=100,
    max_depth=6,
    learning_rate=0.3
    )

# evaluate model
ypred_xgb_pretrained = xgb_pretrained.evaluate(xtest_breast)

In [ ]:
plot_classification_metrics(ytest_breast, ypred_xgb_pretrained, plot_title='XGB Pretrained')

In [ ]:
xgb_results = pd.DataFrame(xgb_pretrained.log, index=[0])
xgb_results

#### OrganCMNIST

In [ ]:
# initialize model
xgb_pretrained = XGBPretrained()

# train model
xgb_pretrained = xgb_pretrained.train(
    x_train=xtrain_organ,
    y_train=ytrain_organ,
    x_val=xval_organ,
    y_val=yval_organ,
    n_estimators=100,
    max_depth=6,
    learning_rate=0.3
    )

# evaluate model
ypred_xgb_pretrained = xgb_pretrained.evaluate(xtest_organ)

In [ ]:
plot_classification_metrics(ytest_organ, ypred_xgb_pretrained, plot_title='XGB Pretrained')

In [ ]:
xgb_results = pd.DataFrame(xgb_pretrained.log, index=[0])
xgb_results

### Concrete

In [ ]:
class ConcretePretrained():

    def __init__(self, n_bits=2):
        self.n_bits = n_bits
        self.model = None
        self.pretrained_model = None
        self.log = {
            'train': 0,
            'compile': None,
            'keygen': None,
            'evaluate_total': None,
            'evaluate_sample': None
        }

    def train(self, x_train, y_train, x_val, y_val, n_estimators=20, max_depth=3, learning_rate=0.1):
        # transform data to tensor
        x_train, x_val = torch.tensor(x_train, dtype=torch.float), torch.tensor(x_val, dtype=torch.float)
        # combine data
        X, y = torch.cat((x_train, x_val), dim=0), np.concatenate((y_train, y_val), axis=0)
        # get resnet50 model
        self.pretrained_model = resnet50(pretrained=True)
        # modify first layer to accept grayscale images with one channel
        self.pretrained_model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        # remove last layer and replace with flattening layer
        self.pretrained_model.fc = torch.nn.Flatten()
        # feature extraction
        with torch.no_grad():
            X = self.pretrained_model(X).detach().numpy()
        # initialize XGBoost model
        self.model = ConcreteXGBClassifier(n_bits=self.n_bits, n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, n_jobs=-1)
        # train model
        start_time = time.time()
        self.model.fit(X, y)
        self.log['train'] = time.time() - start_time
        # compile model
        start_time = time.time()
        self.model.fhe_circuit = self.model.compile(X[:100])
        self.log['compile'] = time.time() - start_time

        return self
    
    def evaluate(self, X, fhe='simulate'):
        # transform data to tensor
        X = torch.tensor(X, dtype=torch.float)
        # feature extraction
        with torch.no_grad():
            X = self.pretrained_model(X).detach().numpy()
        # keygen
        start_time = time.time()
        self.model.fhe_circuit.keygen(force=True)
        self.log['keygen'] = time.time() - start_time
        # predict
        start_time = time.time()
        y_pred = np.array([self.model.predict(X[[i]], fhe=fhe)[0] for i in tqdm(range(X.shape[0]))])
        self.log['evaluate_total'] = time.time() - start_time
        self.log['evaluate_sample'] = self.log['evaluate_total']/X.shape[0]

        return y_pred

#### PneumoniaMNIST

In [ ]:
results = {'n_bits': [], 'y_pred': [], 'times': [], 'metrics': []}

for n_bits in range(2,7):

    concrete_pretrained = ConcretePretrained(n_bits=n_bits)
    concrete_pretrained = concrete_pretrained.train(x_train=xtrain_pneumonia, y_train=ytrain_pneumonia, x_val=xval_pneumonia, y_val=yval_pneumonia, n_estimators=20, max_depth=3, learning_rate=0.1)
    ypred_concrete = concrete_pretrained.evaluate(xtest_pneumonia, fhe=mode)
    metrics = {
        'accuracy':   accuracy_score(ytest_pneumonia, ypred_concrete),
        'precision':  precision_score(ytest_pneumonia, ypred_concrete),
        'recall':     recall_score(ytest_pneumonia, ypred_concrete),
        'f1':         f1_score(ytest_pneumonia, ypred_concrete)
    }

    results['n_bits'].append(n_bits)
    results['y_pred'].append(ypred_concrete)
    results['times'].append(concrete_pretrained.log)
    results['metrics'].append(metrics)

In [ ]:
for preds, bits in zip(results['y_pred'], results['n_bits']):
    print('#################################################################')
    plot_classification_metrics(ytest_pneumonia, preds, plot_title=f'Quantization (n_bits={bits})')

In [ ]:
concrete_results = pd.concat([pd.DataFrame.from_records(results['times']), pd.DataFrame.from_records(results['metrics'])], axis=1)
concrete_results.insert(0, 'n_bits', results['n_bits'])
concrete_results

#### BreastMNIST

In [ ]:
results = {'n_bits': [], 'y_pred': [], 'times': [], 'metrics': []}

for n_bits in range(2,7):

    concrete_pretrained = ConcretePretrained(n_bits=n_bits)
    concrete_pretrained = concrete_pretrained.train(x_train=xtrain_breast, y_train=ytrain_breast, x_val=xval_breast, y_val=yval_breast, n_estimators=20, max_depth=3, learning_rate=0.1)
    ypred_concrete = concrete_pretrained.evaluate(xtest_breast, fhe=mode)
    metrics = {
        'accuracy':   accuracy_score(ytest_breast, ypred_concrete),
        'precision':  precision_score(ytest_breast, ypred_concrete),
        'recall':     recall_score(ytest_breast, ypred_concrete),
        'f1':         f1_score(ytest_breast, ypred_concrete)
    }

    results['n_bits'].append(n_bits)
    results['y_pred'].append(ypred_concrete)
    results['times'].append(concrete_pretrained.log)
    results['metrics'].append(metrics)

In [ ]:
for preds, bits in zip(results['y_pred'], results['n_bits']):
    print('#################################################################')
    plot_classification_metrics(ytest_breast, preds, plot_title=f'Quantization (n_bits={bits})')

In [ ]:
concrete_results = pd.concat([pd.DataFrame.from_records(results['times']), pd.DataFrame.from_records(results['metrics'])], axis=1)
concrete_results.insert(0, 'n_bits', results['n_bits'])
concrete_results

#### OrganMNIST

In [ ]:
results = {'n_bits': [], 'y_pred': [], 'times': [], 'metrics': []}

for n_bits in range(2,7):

    concrete_pretrained = ConcretePretrained(n_bits=n_bits)
    concrete_pretrained = concrete_pretrained.train(x_train=xtrain_organ, y_train=ytrain_organ, x_val=xval_organ, y_val=yval_organ, n_estimators=20, max_depth=3, learning_rate=0.1)
    ypred_concrete = concrete_pretrained.evaluate(xtest_organ, fhe=mode)
    metrics = {
        'accuracy':   accuracy_score(ytest_organ, ypred_concrete),
        'precision':  precision_score(ytest_organ, ypred_concrete, average='macro'),
        'recall':     recall_score(ytest_organ, ypred_concrete, average='macro'),
        'f1':         f1_score(ytest_organ, ypred_concrete, average='macro')
    }

    results['n_bits'].append(n_bits)
    results['y_pred'].append(ypred_concrete)
    results['times'].append(concrete_pretrained.log)
    results['metrics'].append(metrics)

In [ ]:
for preds, bits in zip(results['y_pred'], results['n_bits']):
    print('#################################################################')
    plot_classification_metrics(ytest_organ, preds, plot_title=f'Quantization (n_bits={bits})')

In [ ]:
concrete_results = pd.concat([pd.DataFrame.from_records(results['times']), pd.DataFrame.from_records(results['metrics'])], axis=1)
concrete_results.insert(0, 'n_bits', results['n_bits'])
concrete_results